# Multi-Target Visibility

In [ ]:
# Parameters cell. Set defaults here
import datetime
time = datetime.datetime.fromisoformat('2025-05-01T23:59:00Z')

In [ ]:
import numpy as np
import pandas as pd

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time

from astroplan import Observer, FixedTarget
from astroplan.plots import plot_airmass, plot_sky

import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Target Summary

In [ ]:
targets = [
    FixedTarget(SkyCoord(225.00, -40.00, unit='deg', frame='icrs'), name='Rubin_SV_225_-40'),
    FixedTarget(SkyCoord(250.00, 2.00, unit='deg', frame='icrs'), name='Rubin_SV_250_2'), # Only considered because in the north
    FixedTarget(SkyCoord(280.00, -48.00, unit='deg', frame='icrs'), name='Rubin_SV_280_-48'),
    FixedTarget(SkyCoord(300.00, -41.00, unit='deg', frame='icrs'), name='Rubin_SV_300_-41'),
    FixedTarget(SkyCoord(289.4, -20.2, unit='deg', frame='icrs'), name='New_Horizons'),
]

# Sort by RA
targets.sort(key=lambda _: _.coord.ra.deg, reverse=False)

In [ ]:
time = Time(time)
observer = Observer.at_site('LSST')

In [ ]:
ra = np.array([_.coord.icrs.ra.value for _ in targets])
dec = np.array([_.coord.icrs.dec.value for _ in targets])
name = [_.name for _ in targets]

df = pd.DataFrame(data = {'name': name, 'ra': ra, 'dec': dec})
df

## Basic Information

In [ ]:
time_midnight = observer.midnight(time, which='nearest')
time_sunset = observer.sun_set_time(time_midnight, which='previous')
time_sunrise = observer.sun_rise_time(time_midnight, which='next')

if observer.moon_altaz(time_midnight).alt.value > 0.:
    time_moonrise = observer.moon_rise_time(time_midnight, which='previous')
    time_moonset = observer.moon_set_time(time_midnight, which='next')
else:
    time_moonrise = observer.moon_rise_time(time_midnight, which='next')
    time_moonset = observer.moon_set_time(time_midnight, which='previous')

print('Time (UTC) =', time)
print('Time Evening Civil Twilight (UTC) =', observer.twilight_evening_civil(time_midnight, which='previous').iso)
print('Time Evening Nautical Twilight (UTC) =', observer.twilight_evening_nautical(time_midnight, which='previous').iso)
print('Time Evening Astronomical Twilight (UTC) =', observer.twilight_evening_astronomical(time_midnight, which='previous').iso)
print('Time Midnight (UTC) =', time_midnight.iso)
print('Time Morning Astronomical Twilight (UTC) =', observer.twilight_morning_astronomical(time_midnight, which='next').iso)
print('Time Morning Nautical Twilight (UTC) =', observer.twilight_morning_nautical(time_midnight, which='previous').iso)
print('Time Morning Civil Twilight (UTC) =', observer.twilight_morning_civil(time_midnight, which='previous').iso)
print('Moon Illumination =', observer.moon_illumination(time))
print('Moonrise time (UTC) =', time_moonrise.iso)
print('Moonset time (UTC) =', time_moonset.iso)

In [ ]:
time_array = time_sunset + np.arange(0., (time_sunrise - time_sunset).to(u.hr).value, 0.1) * u.hr

## Visibility on Specified Night

In [ ]:
def plotMoon(time):
    time_sun_set = observer.sun_set_time(time, which='previous')
    time_sun_rise = observer.sun_rise_time(time, which='next')
    if observer.moon_altaz(time_sun_set).alt.value > 0.:
        print('Moon is up at start of night')
        time_moonset = observer.moon_set_time(time_sun_set, which='next')
        plt.axvspan(
            time_sun_set.plot_date, 
            min(time_moonset, time_sun_rise).plot_date, 
            color='yellow', alpha=0.2, zorder=-999,
        )
    if observer.moon_altaz(time_sun_rise).alt.value > 0.:
        print('Moon is up at end of night')
        time_moonrise = observer.moon_rise_time(time_sun_rise, which='previous')
        plt.axvspan(
            max(time_moonrise, time_sun_set).plot_date,
            time_sun_rise.plot_date,
            color='yellow', alpha=0.2, zorder=-999,
        )

In [ ]:
plt.figure()
plt.clf()
ax = plot_airmass(targets, observer, time, brightness_shading=True, altitude_yaxis=True)
plotMoon(time)
ax.legend(loc='lower right', title='Moon Illumination = %.2f'%(observer.moon_illumination(time)))
plt.show()

In [ ]:
plt.figure()
for target in targets:
    plot_sky(target, observer, time_array)
plt.legend(loc='lower right')
plt.show()

## Visibility over the Next Month

In [ ]:
time_plus_month = time + 30 * u.day

plt.figure()
plt.clf()
ax = plot_airmass(targets, observer, time_plus_month, brightness_shading=True, altitude_yaxis=True)
plotMoon(time_plus_month)
ax.legend(loc='lower right', title='Moon Illumination = %.2f'%(observer.moon_illumination(time_plus_month)))
plt.show()

In [ ]:
time_next_month = time + np.arange(0., 30., 1.,) * u.day

lunar_illumination_next_month = observer.moon_illumination(time_next_month)

In [ ]:
x_ticks = Time(np.floor(time_next_month.mjd), format='mjd')
x_tick_labels = np.array([_.iso.split()[0] for _ in x_ticks])

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.axhspan(0., 20., alpha=0.2, color='black')

for target in targets:
    lunar_separation_next_month = observer.moon_altaz(time_next_month).separation(observer.altaz(time_next_month, target.coord)).deg
    ax1.plot(time_next_month.mjd, lunar_separation_next_month, label=target.name)

ax1.set_ylabel('Lunar Separation (deg)')
ax1.set_xticks(x_ticks.mjd[::3], x_tick_labels[::3], rotation=45.)
ax1.set_ylim(0., 180.)

ax2.plot(time_next_month.mjd, lunar_illumination_next_month, c='black', ls='--')
ax2.set_ylabel('Lunar Illumination')
ax2.set_ylim(0.,1.)

ax1.legend(loc='upper center')

fig.tight_layout()